In [ ]:
import importlib
from mkids import TopSoc
import numpy as np
import matplotlib.pyplot as plt
import Scan
import pickle
soc = TopSoc()

In [ ]:
print("ADCs ", soc.adcs)
print("DACs ", soc.dacs)

In [ ]:
adcKey = list(soc.adcs.keys())[0]
fsAdc = soc.adcs[adcKey]['fs']
dacKey = list(soc.dacs.keys())[0]
# Here is that call to configure for the pfb out:
# self.pfb_out.configure(self.dacs[self.dacChannel]['fs']/4, self.mixer, self.dacChannel)
fsDac = soc.dacs[dacKey]['fs']/4
print("fsAdc, fsDac =",fsAdc, fsDac)
fnAdc = fsAdc/2
fnDac = fsDac/2
print("fnAdc, fnDac =",fnAdc, fnDac)



In [ ]:
def fMeasFromFGen(fn, fGen, nZone):
    if np.mod(nZone,2):
        #print(" ODD nZone =",nZone)
        fMeas = (nZone-1)*fn + fGen
    else:
        #print("EVEN nZone =",nZone)
        fMeas = nZone*fn - fGen
    return fMeas

def fGenFromFMeas(fn, fMeas):
    nZone = int(fMeas/fn) + 1
    if np.mod(nZone,2):
        #print(" ODD nZone =",nZone)
        fGen = fMeas - (nZone-1)*fn
    else:
        #print("EVEN nZone =",nZone)
        fGen = nZone*fn - fMeas
    return fGen

In [ ]:
fn = fnDac
fGens = np.linspace(0,fn, 100)
for nZone in [1,2,3,4]:
    fMeas = np.zeros(len(fGens))
    for i,fGen in enumerate(fGens):
        fMeas[i] = fMeasFromFGen(fn, fGen, nZone)
    plt.plot(fMeas,fGens, label="nZone=%d"%nZone)
plt.legend()
fMeass = np.linspace(0,nZone*fn, 18)
fGens = np.zeros(len(fMeass))
for i,fMeas in enumerate(fMeass):
    fGens[i] =  fGenFromFMeas(fn, fMeas)
plt.plot(fMeass, fGens, 'ko')
_ =plt.title("DAC fs = %f"%fsDac)

In [ ]:
fn = fnAdc
fGens = np.linspace(0,fn, 100)
for nZone in [1,2,3,4,5,6,7,8]:
    fMeas = np.zeros(len(fGens))
    for i,fGen in enumerate(fGens):
        fMeas[i] = fMeasFromFGen(fn, fGen, nZone)
    plt.plot(fMeas,fGens, label="nZone=%d"%nZone)
plt.legend()
fMeass = np.linspace(0,nZone*fn, 18)
fGens = np.zeros(len(fMeass))
for i,fMeas in enumerate(fMeass):
    fGens[i] =  fGenFromFMeas(fn, fMeas)
plt.plot(fMeass, fGens, 'ko')
_ =plt.title("ADC fs = %f"%fsAdc)

In [ ]:
scan = Scan.Scan(soc)

In [ ]:
fMixer = 14024
soc.set_mixer(fMixer)

In [ ]:

print("output")
print("fsDac =",fsDac)
print("soc.fsOut =",soc.fsOut)
print("soc.pf_out.fs =",soc.pfb_out.fs)
print("input")
print("fsAdc =",fsAdc)
print("soc.fsIn =",soc.fsIn)

In [ ]:
# Let's set fMixer to be the input Nyquist frequency minus 1/4 ouput bandwidth
outputBandwidth = soc.nOutCh*soc.fcOut
print("outputBandwidth =",outputBandwidth)
inputNyquist = soc.fsIn/2
print("inputNyquist =",inputNyquist)
fMixer = soc.fsIn/2 - outputBandwidth/4
print("fMixer = ",fMixer)
soc.set_mixer(fMixer)

In [ ]:
soc.set_mixer(fMixer)
fTone = fMixer + 12.345
outCh = soc.outFreq2ch(fTone)
inCh = soc.inFreq2ch(fTone)
print("fMixer=%.2f fTone=%.2f outCh=%d inCh=%d"%(fMixer, fTone, outCh, inCh))

In [ ]:
importlib.reload(Scan)
scan = Scan.Scan(soc)
def setAndRead(fTone, average=False, verbose=False):
    scan.setTones(np.array([fTone]), np.array([0.9]), np.array([0.0]))
    scan.prepRead(decimation=2)
    xs = scan.readAndUnpack(nt=1, nsamp=10000, iBegin=1000, average=average)
    if verbose:
        f = scan.toneFreqs[0]
        outCh = scan.outChs[0]
        outDds = scan.outDds[0]
        inCh = scan.inChs[0]
        inOffset = scan.inOffsets[0]
        print(" f=%8.3f  outCh=%3d  outDds=%+6.3f  inCh=%4d   inOffset=%+6.3f"%(fTone, outCh, outDds, inCh, inOffset))
    return xs
x = setAndRead(fTone, verbose=True)


In [ ]:
freqs = 1805.5+0.1*np.arange(30)
for f in freqs:
    x = setAndRead(f, verbose=True)

In [ ]:
plt.plot(np.real(x[0][0]))
plt.plot(np.imag(x[0][0]))
plt.title("fMix=%.2f fTone=%.2f"%(scan.soc.get_mixer(), fTone))

In [ ]:
freqs = 1805.5+0.1*np.arange(30)
amps = np.zeros(len(freqs))
for i,freq in enumerate(freqs):
    x = setAndRead(freq, verbose=True)
    amp = np.abs(x[0][0].mean())
    amps[i] = amp

In [ ]:
plt.plot(freqs,amps, '-o')
#plt.axvline(soc.fsIn/2, c='r')

In [ ]:
freqs = 2047+0.1*np.arange(30)
amps = np.zeros(len(freqs))
for i,freq in enumerate(freqs):
    x = setAndRead(freq, verbose=True)
    amp = np.abs(x[0][0].mean())
    amps[i] = amp

In [ ]:
plt.plot(freqs,amps, '-o')
plt.axvline(soc.fsIn/2, c='r')


In [ ]:
freqs = 2047+0.1*np.arange(100)
amps = np.zeros(len(freqs))
for i,freq in enumerate(freqs):
    x = setAndRead(freq, verbose=True)
    amp = np.abs(x[0][0].mean())
    amps[i] = amp
plt.plot(freqs,amps, '-o')
plt.axvline(soc.fsIn/2, c='r')


In [ ]:
freqs = 2045+2*np.arange(100)
amps = np.zeros(len(freqs))
for i,freq in enumerate(freqs):
    x = setAndRead(freq, verbose=True)
    amp = np.abs(x[0][0].mean())
    amps[i] = amp
plt.plot(freqs,amps, '-o')
plt.axvline(soc.fsIn/2, c='r')
